In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jan 31 12:54:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

### Setup and Load dataset
### Food.com   Dataset
- https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions
- https://aclanthology.org/D19-1613/



In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np

In [ ]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
!gdown --id 1EXgEkJ-rNhHe908ozOfkcWy3LXqnC7Tj

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1EXgEkJ-rNhHe908ozOfkcWy3LXqnC7Tj
To: /content/food.zip
100% 280M/280M [00:03<00:00, 71.1MB/s]


In [ ]:
!unzip -o "food.zip"  -d  "/content"

Archive:  food.zip
  inflating: /content/PP_recipes.csv  
  inflating: /content/PP_users.csv   
  inflating: /content/RAW_interactions.csv  
  inflating: /content/RAW_recipes.csv  
  inflating: /content/ingr_map.pkl   
  inflating: /content/interactions_test.csv  
  inflating: /content/interactions_train.csv  
  inflating: /content/interactions_validation.csv  


In [ ]:
interactions_df = pd.read_csv('interactions_train.csv')

interactions_df.head(1)

,user_id,recipe_id,date,rating,u,i
0,2046,4684,2000-02-25,5.0,22095,44367


In [ ]:
recipes_df = pd.read_csv('PP_recipes.csv')

recipes_df.head(1)

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"


In [ ]:
user_df = pd.read_csv('PP_users.csv')

user_df.head(1)

,u,techniques,items,n_items,ratings,n_ratings
0,0,"[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...","[1118, 27680, 32541, 137353, 16428, 28815, 658...",31,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...",31


In [ ]:
raw_interactions_df = pd.read_csv('RAW_interactions.csv')
raw_interactions_df['review']= raw_interactions_df['review'].astype(str)
raw_interactions_df.head(1)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...


In [ ]:
raw_recipes_df = pd.read_csv('RAW_recipes.csv')

raw_recipes_df.head(1)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7


In [ ]:
raw_interactions_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132367 entries, 0 to 1132366
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id    1132367 non-null  int64 
 1   recipe_id  1132367 non-null  int64 
 2   date       1132367 non-null  object
 3   rating     1132367 non-null  int64 
 4   review     1132367 non-null  object
dtypes: int64(3), object(2)
memory usage: 43.2+ MB


In [ ]:
food_df=interactions_df
food_df.rename(columns = {str("recipe_id"):"itemID"}, inplace = True)
food_df.rename(columns = {str("user_id"):"userID"}, inplace = True)
food_df.rename(columns = {str("rating"):"rating"}, inplace = True)
food_df['date']=food_df['date'].astype(str)
foods_users=len(food_df.userID.unique())
foods_items=len(food_df.itemID.unique())
food_df.shape[0]


698901

In [ ]:
import time
import datetime
def datetimestamp(s):
    return int(datetime.datetime.strptime(s, '%Y-%m-%d').strftime("%s"))

food_df=food_df[['itemID','userID','rating','date']]
food_df=food_df[food_df['rating']>=1]

y = food_df.groupby('userID').count()['rating']>=50
foods = y[y].index
food_df = food_df[food_df['userID'].isin(foods)]
food_df.shape
y = food_df.groupby('itemID').count()['rating']>=20
foods = y[y].index
food_df = food_df[food_df['itemID'].isin(foods)]
food_df['time'] = food_df.apply(lambda row : datetimestamp(row['date']
                     ), axis = 1)
final_foods_users=len(food_df.userID.unique())
final_foods_items=len(food_df.itemID.unique())
food_df.shape[0]


86505

In [ ]:
unique_uesrs=list(food_df.userID.unique())
unique_items=list(food_df.itemID.unique())

In [ ]:
contant_df=raw_recipes_df[raw_recipes_df.id.isin(unique_items)].reset_index(drop=True)

In [ ]:
item_ids = list(set(list(food_df.itemID.unique())))
user_ids = list(set(list(food_df.userID.unique())))
dict_users = {}
index = 0
for ids in sorted(user_ids):
    dict_users[ids] = index
    index += 1
dict_items = {}
index = 0
for ids in sorted(item_ids):
    dict_items[ids] = index
    index += 1
food_df['userID'] = food_df.userID.map(dict_users)
food_df['itemID'] = food_df.itemID.map(dict_items)


In [ ]:

contant_df['id'] = contant_df.id.map(dict_items)

In [ ]:
unique_uesrs=list(food_df.userID.unique())
unique_items=list(food_df.itemID.unique())

# Details about dataset

In [ ]:
summary_df=pd.DataFrame(columns=['Dataset','Algorithm','Sparsity','Precision@k'])

In [ ]:
userID="userID"
itemID="itemID"
score="rating"
timestamp='time'

# Content based filtering

In [ ]:

contant_df['NewTag']= contant_df['ingredients'].str.replace(' ', '', regex=True) +contant_df['tags'].str.replace(' ', '_', regex=True) +contant_df['description'].str.replace(' ', '_', regex=True) 


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
cv = TfidfVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(contant_df['NewTag'].values.astype('str') ).toarray()
cosine_similarity = cosine_similarity(vector)

In [ ]:
def RecommendFood(food,contant_df,similarity):
    index  = contant_df[contant_df['id'] == food].index [0]
    #print(index)
    distance = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    result=[]
    for i in distance[1:11]:
      
        result.append(list(contant_df.iloc[i[0]])[0])
    return result    

In [ ]:
def RecommendFoodID(food,contant_df,similarity):
    index  = contant_df[contant_df['id'] == food].index [0]
    #print(index)
    distance = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    result=[]
    for i in distance[1:11]:
      
        result.append(list(contant_df.iloc[i[0]])[1])
    return result    

In [ ]:
contant_df[contant_df['id'] == unique_items[1]]

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,NewTag
1697,roast sticky chicken,57,325,6411,2001-04-20,"['weeknight', 'time-to-make', 'course', 'main-...","[238.0, 24.0, 9.0, 67.0, 35.0, 22.0, 2.0]",12,"['blend all spices together and set aside', 'r...","beautiful and delicious, this incredibly moist...","['salt', 'paprika', 'cayenne pepper', 'onion p...",10,"['salt','paprika','cayennepepper','onionpowder..."


In [ ]:
resuls=RecommendFood(unique_items[1],contant_df,cosine_similarity)
resuls

['crock pot whole chicken',
 'tortured chicken   beer can',
 'unbelievable baked buffalo wings',
 'deli rotisserie chicken',
 'mermaid s tender roast chicken',
 'hooter s buffalo wings',
 'chipotle pork roast',
 'company pork roast',
 'chicken breasts in lemon cream sauce',
 'incredible oven fried chicken']

In [ ]:
resuls=RecommendFoodID(unique_items[1],contant_df,cosine_similarity)
resuls

[541, 260, 1808, 1791, 1003, 50, 323, 330, 772, 99]